# FF Convergence

In [1]:
import sys
sys.path.insert(0, '../../../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import tensorflow as tf
import random
import pickle as pkl

from qiskit.quantum_info import Operator, Statevector, random_unitary
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm
from math import ceil

from quantum_tools import circuit_to_matrix, partial_trace
from kraus_channels import isomery_to_kraus, KrausMap
from quantum_channel import ConcatChannel, DilutedChannel, IdentityChannel, ChoiMapStatic
from utils import numberToBase, kron
from spectrum import complex_spacing_ratio, channel_spectrum, Spectrum, mean_spacing, coat_spectrum
from analysis import spectrum_to_angular, spectrum_to_radial, hist_ensamble
from qiskit.circuit.library import iSwapGate, XGate, YGate, HGate, CXGate, RGate, RZGate
from quantum_circuits import integrable_circuit, nonintegrable_circuit, haar_random
from synthetic_data import generate_map_data
from optimization import ModelQuantumMap, Logger, model_saver
from loss_functions import ProbabilityMSE, ProbabilityRValue, channel_fidelity_loss

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=1)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


In [2]:
def sample_traceout_csr(n, n_anc, reps, circuit_type, L):
    channel_list = []
    spectra_list = []
    csr_list = []


    for _ in tqdm(range(reps)):
        circuit = circuit_type(n+n_anc, L)
        U = Operator(circuit).data

        kraus_map = isomery_to_kraus(U, rank = 2**n_anc)
        channel_list.append(kraus_map)
        
        spectrum = channel_spectrum(kraus_map)
        spectra_list.append(spectrum)
        csr = complex_spacing_ratio(spectrum, log=False)
        csr_list.append(csr)

    return  channel_list, spectra_list, csr_list

## Free Fermion

### Generate Benchmarks

In [3]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

n = 4
n_anc = 1
d = 2**n
reps = 10

L = 5

channel_FF_list, spectra_FF_list, csr_FF_list = sample_traceout_csr(n, n_anc, reps, integrable_circuit, L)

pkl.dump([channel_FF_list, spectra_FF_list, csr_FF_list], open("data/FF_synthetic_benchmark.pkl", "wb"))

  0%|          | 0/10 [00:00<?, ?it/s]

### Generate Synthetic Data and Fit

In [24]:
model_list = []

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

for channel in channel_FF_list:
    inputs_map, targets_map = generate_map_data(channel, N_map=5000-6**n, shots=None)

    model = ModelQuantumMap(channel = KrausMap(d = d, 
                                        rank = d**2,
                                        ),
                    loss_function = ProbabilityMSE(),
                    optimizer = tf.optimizers.Adam(learning_rate=0.01),
                    logger = Logger(loss_function_list = [ProbabilityRValue(), channel_fidelity_loss], sample_freq=100),
                )
    
    model.train(inputs = inputs_map, 
                targets = targets_map,
                inputs_val = [inputs_map, None],
                targets_val = [targets_map, [channel]],
                N=500,
                num_iter = 3000,
                verbose = True,)
    
    model_list.append(model)

model_saver(model_list, "models/model_list_FF_ideal.pkl")


  0%|          | 0/3000 [00:00<?, ?it/s]

[-0.0026462393549795404, -0.007639048213537014]
[0.5415459781661054, -0.11946888817127664]
[0.883143316531407, -0.4772495917361663]
[0.9731770453506875, -0.733802955880813]
[0.99093442398695, -0.8421032862142404]
[0.9959547979606564, -0.8936690699341776]
[0.9978597243629138, -0.922324946835465]
[0.9987320252526853, -0.9400593998343588]
[0.9991906471130628, -0.9520371289300095]
[0.9994534976350293, -0.9605749376120886]
[0.9996144704313044, -0.9668654088947781]
[0.9997197004860787, -0.9717167708482413]
[0.9997906873079918, -0.9755527657138816]
[0.9998401942634596, -0.9786275307109881]
[0.9998755392929164, -0.9811494067781631]
[0.9999017119323933, -0.9832369348409344]
[0.9999211256635204, -0.9849884330770872]
[0.9999358972198418, -0.9864811514793811]
[0.9999474882801572, -0.9877685769586314]
[0.9999565813283983, -0.9888768311779461]
[0.999963809600754, -0.989849430506745]
[0.9999695953548942, -0.9907024966816499]
[0.9999743847507673, -0.9914588386362343]
[0.9999782609950224, -0.9921300519

  0%|          | 0/3000 [00:00<?, ?it/s]

[0.004428037766317061, -0.008694958956112601]
[0.5726617113889495, -0.1418756325010198]
[0.9036945893038483, -0.5207227572384953]
[0.9788330906390448, -0.7660687264975056]
[0.9928506825492804, -0.863060800036754]
[0.9967872978556466, -0.9080705389349918]
[0.9982942773928758, -0.9330302448805765]
[0.9989876483186725, -0.9484652415233473]
[0.999352587138296, -0.9587983103682327]
[0.9995626181469415, -0.9661812902871277]
[0.9996917924993761, -0.9716452701315975]
[0.9997753046862288, -0.9758358064658509]
[0.9998321959713871, -0.9791190424159963]
[0.9998715965802207, -0.9817547780667127]
[0.9999000137004317, -0.9839158207593449]
[0.9999210270403085, -0.9857157340455338]
[0.9999366110474592, -0.9872253144630716]
[0.9999486109128033, -0.9885112569502134]
[0.999957941138648, -0.9896069683380039]
[0.9999651699378544, -0.9905590065813614]
[0.9999709912511813, -0.9913892100207736]
[0.9999756452505943, -0.9921171922048154]
[0.9999794489385417, -0.9927609133034725]
[0.9999825370588367, -0.993338167

  0%|          | 0/3000 [00:00<?, ?it/s]

...

[0.0026041154124630816, -0.007012077419987845]
[0.5402558540167555, -0.1316551788161232]
[0.9087486072651101, -0.5062827074033983]
[0.978836902170569, -0.7375638398310579]
[0.9921388191645888, -0.8384335342317502]
[0.996374042862989, -0.8900125990828138]
[0.998062807416309, -0.9195982520234123]
[0.9988512721345849, -0.9381038500125555]
[0.9992675558208565, -0.9506204600876486]
[0.9995053982510888, -0.9594567904307194]
[0.9996519341042418, -0.9660254759538001]
[0.9997468259686636, -0.9710593479290711]
[0.9998103724597047, -0.9749716809265079]
[0.9998547040587751, -0.9781337928170382]
[0.9998869283636493, -0.980718117392649]
[0.999910421567252, -0.9828557372120992]
[0.9999280546217564, -0.9846576375168948]
[0.9999416699267897, -0.9861935946281281]
[0.9999521754932563, -0.9875088561405668]
[0.9999604259459559, -0.9886421904378963]
[0.9999669787281708, -0.9896352081895307]
[0.9999721700321924, -0.9905094228216508]
[0.9999764694209798, -0.9912786198960095]
[0.9999800275101519, -0.9919630957

  0%|          | 0/3000 [00:00<?, ?it/s]

[-0.002571590829813619, -0.007255300871313967]
[0.54880273179502, -0.1287869855809339]
[0.8997348799823288, -0.49664969505934226]
[0.9779694506277766, -0.7464202043726825]
[0.9924826997785922, -0.849364019544966]
[0.996609148144908, -0.8982757982133569]
[0.9981963324318303, -0.9256869068910666]
[0.9989329898661439, -0.9428054073879257]
[0.9993176215559952, -0.9542826905207259]
[0.9995393944574195, -0.9624292112059308]
[0.9996755277552565, -0.9684763703592824]
[0.9997636692303286, -0.9731112406443945]
[0.9998232834010995, -0.9767623205203692]
[0.999864635520531, -0.9796865446694809]
[0.9998947348862832, -0.982085319525428]
[0.999916719320955, -0.9840765990534753]
[0.9999331138864219, -0.9857436828447704]
[0.9999457593135281, -0.987167895219108]
[0.9999554985204133, -0.9883915996929862]
[0.9999632300756756, -0.9894528065202873]
[0.9999692743591048, -0.9903758218517524]
[0.9999742326200883, -0.9911866373209651]
[0.9999782182257922, -0.9919045547652391]
[0.999981399078847, -0.9925429144866

  0%|          | 0/3000 [00:00<?, ?it/s]

[-0.000702383799513262, -0.007581216905399038]
[0.557616470550919, -0.1371280869677343]
[0.903637801921684, -0.5294568968744648]
[0.9785621962708984, -0.7679487071046]
[0.9925313657837849, -0.8616015679140532]
[0.9965827781013255, -0.9061507599706088]
[0.9981694041275261, -0.9312673647661877]
[0.9989062750098742, -0.9468655194321126]
[0.999297701845113, -0.9574194106251214]
[0.9995238800663958, -0.9649713310510567]
[0.9996636058932543, -0.9705764721895158]
[0.999754804005096, -0.9748876903234641]
[0.9998161054102772, -0.9782652888454723]
[0.999859226627514, -0.9809982686559825]
[0.9998903144776764, -0.9832293343889668]
[0.9999131103160456, -0.9850878706162002]
[0.9999303440518266, -0.9866597925247478]
[0.999943431279319, -0.9879925535657854]
[0.9999536830740592, -0.9891383336427818]
[0.9999618117151036, -0.9901309231656226]
[0.999968160731168, -0.9909976692304842]
[0.999973264881283, -0.9917573905044629]
[0.9999774113491062, -0.9924325184248257]
[0.9999808269242719, -0.9930299104613555

## Chaotic Haar Random

### Generate Benchmarks

In [25]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

n = 4
n_anc = 1
d = 2**n
reps = 10

channel_FF_list, spectra_FF_list, csr_FF_list = sample_traceout_csr(n, n_anc, reps, haar_random, None)

pkl.dump([channel_FF_list, spectra_FF_list, csr_FF_list], open("chaotic_synthetic_benchmark.pkl", "wb"))

  0%|          | 0/10 [00:00<?, ?it/s]

### Generate Synthetic Data and Fit

In [26]:
model_list = []

for channel in channel_FF_list:
    inputs_map, targets_map = generate_map_data(channel, N_map=5000-6**n, shots=None)

    model = ModelQuantumMap(channel = KrausMap(d = d, 
                                        rank = d**2,
                                        ),
                    loss_function = ProbabilityMSE(),
                    optimizer = tf.optimizers.Adam(learning_rate=0.01),
                    logger = Logger(loss_function_list = [ProbabilityRValue(), channel_fidelity_loss], sample_freq=100),
                )
    
    model.train(inputs = inputs_map, 
                targets = targets_map,
                inputs_val = [inputs_map, None],
                targets_val = [targets_map, [channel]],
                N = 500,
                num_iter = 3000,
                verbose = True,)
    
    model_list.append(model)

model_saver(model_list, "models/model_list_chaotic_ideal.pkl")


  0%|          | 0/3000 [00:00<?, ?it/s]

[-0.0036414200857965984, -0.007972496225420064]
[0.5071657514923241, -0.16586375072042353]
[0.9020300397439713, -0.5974239440542977]
[0.9794143103224303, -0.8119949321393507]
[0.9931468504176384, -0.8909601901340021]
[0.9969589698153629, -0.9272357792914889]
[0.9983983695639304, -0.9471630062439489]
[0.9990592327275677, -0.9595103086001927]
[0.9994029097043269, -0.9677691454893765]
[0.999599334231163, -0.9736039390144732]
[0.9997190100859225, -0.9779118373370363]
[0.9997965303990098, -0.9812047590411053]
[0.9998480940575681, -0.9837823187978573]
[0.9998841706936633, -0.9858479240965041]
[0.9999099699928616, -0.987531006998841]
[0.9999288572128749, -0.9889279005107396]
[0.9999430418309675, -0.9900992876597396]
[0.9999538963707578, -0.9910937596035921]
[0.9999622006761909, -0.9919479891688165]
[0.9999687851590827, -0.9926863648868458]
[0.9999740127075094, -0.9933290471569941]
[0.999978142134678, -0.9938904589821387]
[0.9999815565499101, -0.9943856978064062]
[0.9999842618828779, -0.994826

  0%|          | 0/3000 [00:00<?, ?it/s]

[-0.003843744153595674, -0.008203443048447754]
[0.5106221719924422, -0.1736630949483177]
[0.9068308839157564, -0.6125651948067495]
[0.9804515303129412, -0.8191348883913542]
[0.9934385151124795, -0.8946376200855365]
[0.9970727720667603, -0.9294607036514152]
[0.9984562106940452, -0.948725151973553]
[0.999090153334381, -0.9606330471978942]
[0.9994219908428489, -0.9686193961019177]
[0.9996116483129202, -0.9742853025731886]
[0.9997277432821371, -0.9784716144094104]
[0.9998026710516882, -0.9816776311457608]
[0.9998527511197121, -0.9841932113521832]
[0.9998879782210583, -0.986209981092664]
[0.9999129295180917, -0.9878516280341421]
[0.9999311706633764, -0.9892118400652066]
[0.9999449533259559, -0.990356264387689]
[0.9999554208551268, -0.9913258804166636]
[0.9999634887973852, -0.9921559611827392]
[0.9999698608328003, -0.992874042505037]
[0.9999748651529158, -0.9934967109619914]
[0.9999788888701904, -0.994046959773935]
[0.9999821483428506, -0.9945313050192475]
[0.9999848669772984, -0.99496306225

  0%|          | 0/3000 [00:00<?, ?it/s]

...

[-0.0032370680178048783, -0.007777812599940757]
[0.5099703924489898, -0.16989603854784452]
[0.9065869369868075, -0.6097605502808741]
[0.9805460469413286, -0.8185295013212792]
[0.9934867966634582, -0.8944253721574444]
[0.9971068612485952, -0.9294846499246227]
[0.9984735010922289, -0.9487403109614113]
[0.9991023215108398, -0.9606803977122256]
[0.9994301768330902, -0.9686741252692311]
[0.9996173657311528, -0.974341884267772]
[0.9997315291733814, -0.9785170149561846]
[0.9998051444524937, -0.9817087890740451]
[0.99985474451614, -0.9842160944841888]
[0.9998893304984946, -0.9862238450602269]
[0.9999139776456527, -0.987865924126838]
[0.9999320585604228, -0.989222903929021]
[0.9999455752311884, -0.9903643377046818]
[0.9999558988923299, -0.9913337119974813]
[0.9999638979133271, -0.9921649503605137]
[0.9999701501308322, -0.9928809687153151]
[0.9999751386668525, -0.9935075949937209]
[0.9999791401368099, -0.9940552323775232]
[0.9999823952095709, -0.9945407511587564]
[0.9999850093542796, -0.99496908

  0%|          | 0/3000 [00:00<?, ?it/s]

[-0.006531575868886863, -0.00835142144387645]
[0.5090660085442997, -0.1677475671633954]
[0.9034379006710151, -0.602324158276932]
[0.9797911211932192, -0.8144073061163765]
[0.9932467723248543, -0.8920644466944317]
[0.9969959672097208, -0.9278612988253978]
[0.998418775780033, -0.9476147496866326]
[0.9990707036820009, -0.9598351500874542]
[0.9994102326491938, -0.9679989932178675]
[0.9996031950077673, -0.9737656001103607]
[0.9997212611127434, -0.9780282629164146]
[0.9997978048137501, -0.981295444068913]
[0.9998492889716264, -0.9838624393066124]
[0.9998850099763549, -0.9859140325737573]
[0.9999107768463303, -0.9875886130341348]
[0.999929348444819, -0.9889775145610462]
[0.9999434821510963, -0.9901401697770212]
[0.9999541512066591, -0.991129714513499]
[0.9999624779971709, -0.9919775006258323]
[0.999968912232514, -0.992711296026002]
[0.9999741426074582, -0.9933505869078386]
[0.9999782888177401, -0.9939106589211973]
[0.999981630422875, -0.9944049445988925]
[0.9999843870618257, -0.99484335461281

  0%|          | 0/3000 [00:00<?, ?it/s]

[-0.0019597575669443312, -0.007833792105721103]
[0.5068663571808388, -0.17033355575224762]
[0.9046082791327688, -0.6077754959172197]
[0.9801124198716797, -0.8176095248645688]
[0.9933702940498405, -0.8941505729253719]
[0.9970519574581035, -0.9292846707374228]
[0.9984506856532428, -0.9486885146846388]
[0.9990896794749503, -0.9606654625951948]
[0.9994218355252694, -0.9686678097704366]
[0.9996117371055865, -0.9743417716720005]
[0.9997277462826959, -0.9785171939611257]
[0.9998024119006763, -0.981713468486806]
[0.9998526393335985, -0.9842237172097774]
[0.9998876309905653, -0.9862341546021867]
[0.999912805608652, -0.9878762998055786]
[0.9999311363507412, -0.989238387083625]
[0.9999449422391596, -0.9903800276254828]
[0.9999553802385681, -0.9913459362734363]
[0.9999634345319091, -0.9921726307419698]
[0.9999697568219624, -0.9928900826851421]
[0.9999747712909709, -0.9935127856596976]
[0.9999788054031548, -0.9940610224482731]
[0.9999821406499269, -0.9945454620383641]
[0.9999847670578023, -0.994974